<a href="https://colab.research.google.com/github/Morningspread/Angry-Tweeter/blob/main/XGBoost_Twitter_Feb2%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comparison of kNN, logistic regression & random forest

##Imports, display parameters and Google Drive

In [66]:
#Sean's imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import sklearn 
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from matplotlib import pyplot as plt
from sklearn import  metrics
#print("scikit-learn version: %s" %sklearn.__version__)

In [67]:
#Riddhi's imports...

import nltk

nltk.download("vader_lexicon")
nltk.download("punkt")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
#XG Boost now on both computers... 

from xgboost import XGBClassifier
from xgboost import plot_importance

In [69]:
# We want to max out the display here
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [70]:
#Mounting the drive... 

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### We need 4 metrics: accuracy, recall, precision and f1 score

In [71]:
#Some code to compute metrics and display a table
def mymetrics(pred,truth):
    table = metrics.confusion_matrix(pred,truth)
    tp = table[1,1]
    tn = table[0,0]
    fp = table[1,0]
    fn = table[0,1]
    
    accuracy =(tp+tn)/pred.shape[0]
    
    if (tp + fn) == 0:
        recall = 0.0
    else:
        recall = tp/(tp+fn)
    if (tp + fp) == 0:
        precision = 0.0
    else:
        precision = tp/(tp+fp)
    if (recall+precision) == 0.0:
        f1 = 0.0
    else:
        f1 = 2. * recall*precision/(recall+precision)
    
    ## Python code to print the data
    a = tn
    b = fn
    c = a+b
    e = fp
    f = tp
    g = e + f
    #h = a + e
    d = {"Pred-0": [a, b, c],
    "Pred-1": [e, f, g] }
    print ("{:<8} {:<8} {:<20} {:<20}".format(' ','True-0','True-1','rowsum'))
    for k, v in d.items():
        lang, perc, change = v
        print ("{:<8} {:<8} {:<20} {:<20}".format(k, lang, perc, change))
    print(" ")
    print("accuracy =",(format (accuracy, '.2f')),"   recall =",(format (recall, '.2f')),
    "   precision =",(format (precision, '.2f')),"   f1_score =",(format (f1, '.2f')))

## Read and pre-process data set

In [72]:
#Checking the home directory 

import os
cwd = os.getcwd()
cwd

#Changing the directory - for the Workstation 
os.chdir(r'/content/drive/MyDrive/Sean/Twitter/')

#Google Drive/Analytics Rebirth/Projects/Despair/MLElect_Sept15.csv

#Changing the directory for the X1 Yoga 
#os.chdir(r'C:\\Users\\Lenovo\\Google Drive\\Analytics Rebirth\\Despair')

In [73]:
#importing the dataset - careful not to get the meta-million dataset
# 

df = pd.read_csv("/content/drive/MyDrive/Sean/Twitter/COVID_223KTweets.csv")

In [74]:
#Let's look at the columns here...


print(df.shape)

df.head()

(223955, 36)


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1578109493392515073,1578109493392515073,2022-10-06 19:46:37 UTC,2022-10-06,19:46:37,0,1518980837135589376,krystynaklepin1,Christine K.,NaN,SAFE &amp; EFFECTIVE. Pfizer skipped major cat...,en,[],[],[],0,0,0,[],[],https://twitter.com/KrystynaKlepin1/status/157...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1578109461184454656,1578101418317082624,2022-10-06 19:46:30 UTC,2022-10-06,19:46:30,0,1497906018029907970,turdfur58439807,BlackSanta9111,NaN,@montanamike_tx @CastAwayOne1 Its a very small...,en,[],[],['https://pbs.twimg.com/media/FeaR-nlXEBsl6BB....,0,0,1,[],[],https://twitter.com/TurdFur58439807/status/157...,False,NaN,1,https://pbs.twimg.com/media/FeaR-nlXEBsl6BB.jpg,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'montanamike_tx', 'name': 'mi...",NaN,NaN,NaN,NaN
2,1578109458038853637,1578109458038853637,2022-10-06 19:46:29 UTC,2022-10-06,19:46:29,0,36803382,yakimahealth,Yakima Health District,NaN,Escuela Secundaria West Valley Junior High lle...,es,[],['http://YakimaVacunas.org'],['https://pbs.twimg.com/media/FeaSMFDXEA8-GVj....,0,0,0,[],[],https://twitter.com/yakimahealth/status/157810...,False,NaN,1,https://pbs.twimg.com/media/FeaSMFDXEA8-GVj.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1578109438464241666,1578087822858522630,2022-10-06 19:46:24 UTC,2022-10-06,19:46:24,0,16635842,spectrummag,SPECTRUM (Since 1969),NaN,"Lisa Beardsley-Hardy, GC director of education...",en,[],[],['https://pbs.twimg.com/media/FeaSGnfaYAAdYva....,0,0,0,['gcac22'],[],https://twitter.com/spectrummag/status/1578109...,False,NaN,1,https://pbs.twimg.com/media/FeaSGnfaYAAdYva.jpg,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1578109416825839616,1578107391580643329,2022-10-06 19:46:19 UTC,2022-10-06,19:46:19,0,1304242839228997632,yesbutcovid19,הטוויטז של גיא,NaN,@Techslut אה זה ננגד שפיכת זרע לבטלה? לא הבנתי...,iw,[],[],[],0,0,0,[],[],https://twitter.com/yesbutCOVID19/status/15781...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Techslut', 'name': 'Ilana Br...",NaN,NaN,NaN,NaN


##Filter only english tweets

In [75]:
df=df[df['language']=='en']

##Creating hashtag and cashtag variable


In [76]:
#We need to create a hashtag variable...based on zero one...or the number of hashtags... 

df['hashtags'].value_counts()

[]                                                                                                                                                                                                         75343
['covid19']                                                                                                                                                                                                 5804
['covid']                                                                                                                                                                                                    380
['todaysmedicalupdate', 'medicine', 'health', 'patients', 'nhs', 'fmrevolution', 'today', 'insurance', 'pharmaceutical', 'biotech', 'covid19', 'coronavirus', 'doctor', 'policy', 'vaccine', 'medtech']      325
['coronavirus', 'covid19']                                                                                                                                          

In [77]:
#Create hashtag variable

df['hash_count'] = df['hashtags'].str.split(", ").str.len()

In [78]:
#Creating cashtag variable 

df['cash_count'] = df['cashtags'].str.split(", ").str.len()

#Create @handle count


In [79]:
#Creating a function that counts the tweets..

def count_handles(tweet):
    return tweet.count("@")

    # add a new column to the dataframe with the handle count
df["handle_count"] = df["tweet"].apply(count_handles)

In [80]:
#Checking the handle counts variable 

#df['handle_count'].value_counts()

##Create sentiment index



In [81]:
def get_relative_length(text, TWITTER_ALLOWED_CHARS=280.0):
    """
    Returns the relative length of text.

    Parameters:
    ------
    text: (str)
    the input text

    Keyword arguments:
    ------
    TWITTER_ALLOWED_CHARS: (float)
    the denominator for finding relative length

    Returns:
    -------
    relative length of text: (float)
  
    Normalizing the length of tweet
    """
    return len(text) / TWITTER_ALLOWED_CHARS


def get_length_in_words(text):
    """
    Returns the length of the text in words.

    Parameters:
    ------
    text: (str)
    the input text

    Returns:
    -------
    length of tokenized text: (int)

    """
    return len(nltk.word_tokenize(text))


def get_sentiment(text):
    """
    Returns the compound score representing the sentiment: -1 (most extreme negative) and +1 (most extreme positive)
    The compound score is a normalized score calculated by summing the valence scores of each word in the lexicon.

    Parameters:
    ------
    text: (str)
    the input text

    Returns:
    -------
    sentiment of the text: (str)
    """
    scores = sid.polarity_scores(text)
    return scores["compound"]

In [82]:
df.shape

(116687, 39)

In [83]:
#Assign the function to new variables... 
#We are only interested in Sentiment - as of Feb 1, 2023

#df = df.assign(n_words=df["tweet"].apply(get_length_in_words))
df['sentiment'] = df.assign(vader_sentiment=df["tweet"].apply(get_sentiment))
#df = df.assign(rel_char_len=df["tweet"].apply(get_relative_length))

ValueError: ignored

##Create tweet length variable 

In [ ]:
#We want to create the tweet length here...

tweet = df['tweet']

df['tweetlength'] = [len(t.split(' ')) for t in tweet]

In [ ]:
#Let's plot the length in the tweets
#Code syntax: hist = df.hist(bins=3)

hist = df['tweetlength'].hist(bins=10)

print(df['tweetlength'].describe())

## Image y/n and date/time variable 

In [ ]:
df['image_len'] = df.photos.map(len)

In [ ]:
#Create a y/n for images... 

df['imageyn']=np.where(df['image_len'] <= 0, 0,1)

In [ ]:
#But we might have to first get the object variables into datetime 
# Syntax:  pd.to_datetime(s, infer_datetime_format=True)

df['date']= pd.to_datetime(df['date'],infer_datetime_format=True)
df['time']= pd.to_datetime(df['time'],infer_datetime_format=True)

In [ ]:
#Trying to convert the date into an integer... 
#Syntax:  test_df['Date'].dt.strftime("%Y%m%d").astype(int)

df['int_date'] = df['date'].dt.strftime("%Y%m%d").astype(int)
df['int_time'] = df['time'].dt.strftime("%H%M%S").astype(int)

In [ ]:
#Checking the variable types... 

print(df['int_date'].dtype)
print(df['int_time'].dtype)

##Creating variable for unique_id 

Created with ChatGPT Feb 1,2023

In [ ]:
#Let's check the shape of the dataset

df.shape

In [ ]:
#How many unique user_ids do we have? 
#So we have about 50% of the dataset with non-unique IDs...
#How would we get at the number of tweets...? What about a groupby dataset...

df['user_id'].nunique()

In [ ]:
#Code source -> ChatGPT

def create_row_column(df, col_name):
  col = df[col_name]
  row = []
  for i in range(len(col)):
    row.append(i)
  df['new_user_id'] = pd.Series(row)
  return df


In [ ]:
#Calling the function..

df = create_row_column(df, 'user_id')

In [ ]:
df['new_user_id'].isnull().sum()

##Tweet Rate - stuck on the interpretation 
  

In [ ]:
#creating the tweet rate

from datetime import datetime
then = pd.to_datetime(min(df['date']))
now  = pd.to_datetime(max(df['date']))                        
duration = now - then                        
duration_in_seconds = duration.total_seconds()      
duration_in_min=duration_in_seconds/60
duration_in_min

#Can this be applied back to the column? Not really...even though it's a vi

#df['tweet_rate'] = duration_in_min


##Define a new dataset - df shape (116687, 48)

In [ ]:
df.columns

In [ ]:
#So we can use the above data to define our dataset...
#Let's remove hte other engagement metrics for now - outcome below: "retweets_count","likes_count"
#Outcome variable: data['nreplies'] = np.where(data['replies_count'] >= 1, 0,1)
#Let's remove the other Twitter engagement metrics as they are going to be correlated with the outcome: ,"retweets_count","likes_count" 

data = pd.DataFrame(df, columns = ["int_date","int_time","replies_count","user_id","sentiment",
                                   "handle_count","tweetlength","hash_count","cash_count","imageyn"])

#Let's look at the table...

data.head()

#I want to print the shape and see how it compares to nunique
print(data.shape)

#Dealing with missing data

In [ ]:
#Checking for missing data... 

data.isnull().sum()

##Looking at correlational structure

In [ ]:
#Let's have an optional step of checking the correlations

data.corr()

#Creating the target variable 

In [ ]:
#Dichotomizing a variable -> We are losing a lot of information here...but allows us to answer different questions... 

data['nreplies'] = np.where(data['replies_count'] >= 1, 0,1)

data['nreplies'].value_counts()

In [ ]:
#Dropping the data column...because it's gonna be the best predictor... 

data.drop(['replies_count'], axis = 1, inplace=True) 

In [ ]:
# Now let's try creating a target array... let's call it "target" - it HAS to be a categorical variable

target = np.array(data.nreplies)

#Split out the checking step from the creation set, because it gets dropped
target.shape

In [ ]:
#important STEP Drop the Target/Outcome Variable
#The secondary variables that can also be dropped: 'year', 'totalvotes'

data = data.drop(columns = ['nreplies'])

# NOTE: I changed random_state to 0 to get more 1's in X_test

In [ ]:
#This won't split unless our arrays are aligned... 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state = 0)

#Now let's check these - ok, they seemed to be working...  
X_train.shape
#X_test?

In [ ]:
#Testing out the size of the test dataset
y_test.shape

Let's start by looking at an ideal machine learning tool and see what the metrics would be. We'll take the y_train and assume that is our prediction.

In [ ]:
mymetrics(y_train,y_train)

In the above table, we show true negative, false negative, false positive and true positive (with rowsum for each row).
            tn     fn
            fp     tp
So ideally, we should get a table with 0 in the off-diagonal and non-zeros along the diagonal, and all metrics should be equal to 1.

## LOGISTIC REGRESSION - standardization of the data is required for this method

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_std = scaler.fit_transform(X_train)

We have to use class_weight = 'balanced' to account for imbalanced responses

In [ ]:
# Time to use some heavy artillery
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1.0,class_weight='balanced',max_iter=300)

logreg.fit(X_std, y_train)

In [ ]:
Y_pred = logreg.predict(X_std)
print("TRAIN: Logistic Regression")
mymetrics(Y_pred,y_train)

Note that accuracy has dropped but recall looks good. Our goal is to have a high accuracy and a high recall (so precision is going to take a hit). Try the test set but it needs to be standardized too.

In [ ]:
Xt_std = scaler.fit_transform(X_test)

Yt_pred = logreg.predict(Xt_std)
print("TEST Logistic Regression")
mymetrics(Yt_pred,y_test)

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#This is a tree-based method

# train model with "balanced" setting to tell RandomForest that imbalance exists in data set
k = 100
rfc = RandomForestClassifier(n_estimators=k,max_depth=4,criterion='entropy',class_weight='balanced',random_state=11).fit(X_train, y_train)

# predict on train set

rfc_pred = rfc.predict(X_train)

In [ ]:
print("TRAIN: Random Forest n_estimators=100")
mymetrics(rfc_pred,y_train)

In [ ]:
rfc_predt = rfc.predict(X_test)

In [ ]:
print("TEST: Random Forest n_estimators=100")
mymetrics(rfc_predt,y_test)

## LAST STEP IS TO TRY MANY TRAIN/TEST SETS 

To validate that the results are repeatable. Use random_state to select different training/test sets below. Takes about 1 minute to run. Check plots and averages at the end.

In [ ]:
# Selecting good values for various parameters and trying different train/test sets
rec = []
pre = []
acc = []
f1 = []
trec = []
tpre = []
tacc = []
tf1 = []
cm0 = []
cm1 = []
cm00 = []
cm11 = []
for k in range(100):                                                #loop on this    v   
    X_train, X_test, y_train, y_test = train_test_split(data, target, random_state = k)
    #print(k)
    rfc = RandomForestClassifier(n_estimators=100,max_depth=4,class_weight='balanced',random_state=11).fit(X_train, y_train)
    
    Y_pred = rfc.predict(X_train)
    recall = metrics.recall_score(Y_pred,y_train)
    precision = metrics.precision_score(Y_pred,y_train)
    accuracy = metrics.accuracy_score(Y_pred,y_train)
    f1_score = metrics.f1_score(Y_pred,y_train)
    rec.append(recall)
    pre.append(precision)
    acc.append(accuracy)
    f1.append(f1_score)
    cmtr = metrics.confusion_matrix(Y_pred,y_train)
    #cm0.append(cmtr[0,0]/(y_train.shape[0]-np.sum(y_train)))
    #cm1.append(cmtr[1,1]/(np.sum(y_train)))
    cm0.append(cmtr[1,1])
    
    rfc_predt = rfc.predict(X_test)
    recall = metrics.recall_score(rfc_predt,y_test)
    precision = metrics.precision_score(rfc_predt,y_test)
    accuracy = metrics.accuracy_score(rfc_predt,y_test)
    f1_score = metrics.f1_score(rfc_predt,y_test)
    trec.append(recall)
    tpre.append(precision)
    tacc.append(accuracy)
    tf1.append(f1_score)
    cmts = metrics.confusion_matrix(rfc_predt,y_test)
    #cm00.append(cmts[0,0]/(y_test.shape[0]-np.sum(y_test)))
    #cm11.append(cmts[1,1]/(np.sum(y_test)))
    cm00.append(cmts[1,1])
    
print("TRAINING SET metrics")
plt.plot(rec)
plt.plot(pre)
plt.plot(acc)
plt.plot(f1)
plt.show()
print("TEST SET metrics")
plt.plot(trec)
plt.plot(tpre)
plt.plot(tacc)
plt.plot(tf1)
plt.show()
#plt.plot(cm0)
#plt.show()
#plt.plot(cm1)
#plt.plot(cm00)
#plt.plot(cm11)
#plt.show()

In [ ]:
#Accuracy
np.mean(acc)

In [ ]:
#Recall
np.mean(tpre)

In [ ]:
#Precision
np.mean(trec)

In [ ]:
#F1 score
np.mean(tf1)

## Ensemble Method 1: Try bagging (i.e., averaging many runs)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
rfc = RandomForestClassifier(n_estimators=100,max_depth=4,criterion='entropy',class_weight='balanced',random_state=11).fit(X_train, y_train)

clf = BaggingClassifier(rfc,max_samples=1.0, max_features=1.0, bootstrap=False,
                        n_estimators=100, random_state=0).fit(X_train, y_train)
Yt_pred = clf.predict(X_test)

In [ ]:
mymetrics(Yt_pred,y_test)

## Ensemble Method 2: Boosting using xgboost

In [ ]:
#So the clf classifier gets instantiated in this cell 
#When I -> "ask it what is" - clf? Says it's an XGB classifier

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from xgboost import XGBClassifier


clf = XGBClassifier(
        objective = 'binary:logistic', 
        booster = "gbtree",
        eval_metric = 'auc', 
        nthread = 4,
        eta = 0.1,
        reg_lambda = 0.2,
        gamma = 0.4,
        max_depth = 5, 
        subsample = 0.5, 
        scale_pos_weight = 30.0, 
        colsample_bytree = 0.7, 
        colsample_bylevel = 0.7,
        min_child_weight = 2,
        alpha = 0,
        random_state = 30, 
        nrounds = 200,
        n_estimators=100
    )


In [ ]:
clf.fit(X_train,y_train)

In [ ]:
Yt_pred = clf.predict(X_train)


In [ ]:
mymetrics(Yt_pred,y_train)

In [ ]:
Yt_pred = clf.predict(X_test)

In [ ]:
mymetrics(Yt_pred,y_test)

In [ ]:
#Trying another method for XG Boost 

plot_importance(clf)
plt.show()

In [ ]:
# Trying to get at feature importance 
print(clf.feature_importances_)

In [ ]:
#Plotting feature importance
plt.bar(range(len(clf.feature_importances_)), clf.feature_importances_)
plt.show()

In [ ]:
mymetrics(Y_pred,y_train)

In [ ]:
#Trying to get at feature importance 

Yt_pred = clf.predict(X_test)

In [ ]:
mymetrics(Yt_pred,y_test)

In [ ]:
#Let's try another - simpler XG Boost model...

model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
mymetrics(Yt_pred,y_test)